Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Предположим, нам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово (переменная KEYWORDS).

Поиск будем вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге я сформирую датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

Пояснения буду давать либо после ячейки, либо после хэшей. 

Актуальность статей по состоянию на 30.08.2020 г.

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
#записываем в эту переменную требуемые слова, заголовки с которыми мы ищем
KEYWORDS = ['многоразовость', 'Asana', 'Битрикс', 'Google', 'Python']

In [3]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [4]:
posts = soup.find_all('article', class_='post')

Пишу код, который создает датафрем с требуемыми столбцами (в данном случае - с датой, названием и ссылкой), предварительно спарсив эти данные с https://habr.com/ru/all/ с помощью библиотеки requests и метода get. 

In [5]:
habr_news = pd.DataFrame()
for art in posts:
#     print(art)
    arts = art.find('a', class_='post__title_link').text
    title_element = art.find('a', class_='post__title_link').text
    urls = art.find('a', class_='post__title_link').get('href')
    dates = art.find('span', class_='post__time').text
    row = {'date': dates, 'title': title_element, 'url': urls}
    habr_news = pd.concat([habr_news, pd.DataFrame([row])])
habr_news

,date,title,url
0,сегодня в 22:35,Выбор генератора форм для Vue.js,https://habr.com/ru/post/517142/
0,сегодня в 22:16,Как «готовить» nullable reference types с доба...,https://habr.com/ru/post/517102/
0,сегодня в 21:34,FOSS News №31 – дайджест новостей свободного и...,https://habr.com/ru/post/517138/
0,сегодня в 20:54,Белорусский AI сервис опередил Google и Micros...,https://habr.com/ru/post/517136/
0,сегодня в 20:14,Как использовать новейшие функции JavaScript в...,https://habr.com/ru/post/517134/
0,сегодня в 19:57,Многоразовость. Как ракете вернуться целой пос...,https://habr.com/ru/post/517132/
0,сегодня в 19:38,Как мы воспринимаем цвет. Занимательные факты....,https://habr.com/ru/post/473620/
0,сегодня в 18:39,Benchmarking. Введение для начинающих,https://habr.com/ru/post/517128/
0,сегодня в 18:27,Как выбирать программное обеспечение для патчи...,https://habr.com/ru/company/otus/blog/517126/
0,сегодня в 17:47,"Думал, что добьюсь большего к своим 35",https://habr.com/ru/post/517122/


Создаю датафрейм filtered_result, в который добавляю результат работы цикла, который в свою очередть проверяет исходный датафрейм на вхождение слов, заданных в переменной KEYWORDS. 

In [6]:
filtered_result = pd.DataFrame()
for keys in KEYWORDS:
    filtered = habr_news[habr_news['title'].str.contains(keys, case=False)]
#     print(filtered)
    for index, row in filtered.iterrows():
#         print(row)
        df = pd.DataFrame([row])
        filtered_result = pd.concat([filtered_result, df])
filtered_result

,date,title,url
0,сегодня в 19:57,Многоразовость. Как ракете вернуться целой пос...,https://habr.com/ru/post/517132/
0,сегодня в 17:21,Управление разработкой и производством в Asana,https://habr.com/ru/post/517124/
0,сегодня в 16:20,«Только не „Битрикс“!». Почему не стоит игнори...,https://habr.com/ru/post/517114/
0,сегодня в 20:54,Белорусский AI сервис опередил Google и Micros...,https://habr.com/ru/post/517136/


In [7]:
# Или ещё один вариант поиска вхождений: через регулярные выражения. Как видим, результат такой же. 

pat = r'(\b{}\b)'.format('|'.join(KEYWORDS))
habr_news[habr_news["title"].str.contains(pat, case=False, na=False)]

/usr/local/Cellar/jupyterlab/2.1.2/libexec/lib/python3.8/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,date,title,url
0,сегодня в 20:54,Белорусский AI сервис опередил Google и Micros...,https://habr.com/ru/post/517136/
0,сегодня в 19:57,Многоразовость. Как ракете вернуться целой пос...,https://habr.com/ru/post/517132/
0,сегодня в 17:21,Управление разработкой и производством в Asana,https://habr.com/ru/post/517124/
0,сегодня в 16:20,«Только не „Битрикс“!». Почему не стоит игнори...,https://habr.com/ru/post/517114/
